The preprocessing stage was completed on my personal system. As training requires a GPU, I proceeded to train the model on Google Colab and imported all the preprocessed data from Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

In [ ]:
train_images = np.load('/content/drive/MyDrive/Dataset/sen12_compressed/train_images.npy')
train_labels = np.load('/content/drive/MyDrive/Dataset/sen12_compressed/train_labels.npy')

In [ ]:
test_images = np.load('/content/drive/MyDrive/Dataset/sen12_compressed/test_images.npy')
test_labels = np.load('/content/drive/MyDrive/Dataset/sen12_compressed/test_labels.npy')

In [ ]:
#rescaling of train_image(All the images are 16 bit image so max value of each pixel can be 2^16-1 = 65535)
train_images = train_images.astype(np.float32)
train_labels = train_labels.astype(np.int32)

train_images /= 65535.0

In [ ]:
test_images = test_images.astype(np.float32)
test_labels = test_labels.astype(np.int32)

test_images /= 65535.0

In [ ]:
train_images.shape

(1750, 256, 256, 4)

In [ ]:
train_labels.shape

(1750,)

In [ ]:
fourth_channel_train_images = train_images[:, :, :, 3:]
train_images = train_images[:, :, :, :3]

In [ ]:
modified_images = np.repeat(fourth_channel_train_images, repeats=3, axis=3)

In [ ]:
print(modified_images.shape)
print(train_images.shape)

(1750, 256, 256, 3)
(1750, 256, 256, 3)


In [ ]:
(unique, counts) = np.unique(train_labels, return_counts=True)

print(f"The number of images in the train dataset containing flooded areas is {counts[1]}\n")
print(f"While the number of images clean from floods is {counts[0]}")

The number of images in the train dataset containing flooded areas is 394

While the number of images clean from floods is 1356


In [ ]:
from tensorflow.keras import models
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, concatenate
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
# resize_and_rescale = tf.keras.Sequential([
#     # tf.print("hello")
#     tf.keras.layers.Lambda(lambda x: tf.print(x)),
#     #tf.keras.layers.Lambda(lambda x: tf.cast(x, tf.float32)),
#     #tf.keras.layers.experimental.preprocessing.Rescaling(1./65535.0),
# ])

In [ ]:
# Define the input shape
input_shape = (256, 256, 3)

# Load the pre-trained ResNet-152 model without the top (classification) layers
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = True

# Create the input tensors for two images
input_tensor_1 = tf.keras.Input(shape=input_shape)
input_tensor_2 = tf.keras.Input(shape=input_shape)

# Apply the pre-trained ResNet-152 model to the first input tensor
x1 = base_model(input_tensor_1, training=True)

# Apply the pre-trained ResNet-152 model to the second input tensor
x2 = base_model(input_tensor_2, training=True)

# Concatenate the outputs of the two base models
concatenated = concatenate([x1, x2])

# Add your own classification layers on top of the concatenated features
x = GlobalAveragePooling2D()(concatenated)
x = Dense(8000, activation='relu')(x)
x = Dropout(0.5)(x)  # Add a dropout layer with a dropout rate of 0.5
x = Dense(4000, activation='relu')(x)
x = Dropout(0.5)(x)  # Add another dropout layer with a dropout rate of 0.5
x = Dense(2000, activation='relu')(x)
x = Dropout(0.5)(x)  # Add another dropout layer with a dropout rate of 0.5
x = Dense(1000, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
# Create the final model
with tf.device('/GPU:0'):
  model = Model(inputs=[input_tensor_1, input_tensor_2], outputs=predictions)

In [ ]:
# with tf.device('/GPU:0'):
#   model = tf.keras.Sequential([resize_and_rescale, model])

In [ ]:
with tf.device('/GPU:0'):
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model with your dataset
with tf.device('/GPU:0'):
  history = model.fit([train_images, modified_images], train_labels, batch_size=12, epochs=55, validation_split=0.1)

In [ ]:
model.summary()

In [ ]:
model.save('/content/drive/MyDrive/Flood_Dtection_Model/resnet_tl_grey.h')